预训练词向量**SGNS**

# 导包

In [1]:
import pandas as pd
from gensim.models import word2vec
from google.colab import drive
import numpy as np
import csv

In [2]:
N_CLASS = 17

# 读取数据

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/data/track1/track1_round1_train_20210222.csv', sep="\|,\|",header=None,names=['id','text','label'])
test_df = pd.read_csv('/content/drive/MyDrive/data/track1/track1_round1_testA_20210222.csv', sep="\|,\|",header=None,names=['id','text'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [7]:
pre_text=list(train_df['text']) + list(test_df['text'])
with open('/content/drive/MyDrive/data/track1/tmp/text.txt','a',encoding='utf-8') as f:
    for t in pre_text:
        f.write(t+'\n')

# 将多标签转化为one-hot向量

In [5]:
content = []
train_df['label'].fillna(value=-1,inplace=True)
for i,id in enumerate(train_df['id']):
    ob = []
    ob.append(train_df['id'][i])
    ob.append(train_df['text'][i])
    labels = str(train_df['label'][i]).split()
    if labels[0] == '-1':
      a = list(np.zeros(N_CLASS))
    else:
      a = list(np.sum(np.eye(N_CLASS)[[eval(label) for label in labels]], axis=0))
    ob.append(a)
    content.append(ob)

In [6]:
headers = ['id','text','label']
with open('/content/drive/MyDrive/data/track1/tmp/train_total.csv','w')as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(content)

# 训练词向量

In [8]:
text = word2vec.LineSentence('/content/drive/MyDrive/data/track1/tmp/text.txt')
model = word2vec.Word2Vec(text, sg=1, size=50, negative=5, hs=1, seed=1)

# 保存词向量

In [9]:
model.save('/content/drive/MyDrive/data/track1/tmp/MyModel')
model.wv.save_word2vec_format('/content/drive/MyDrive/data/track1/tmp/mymodel.txt', binary=False)